In [13]:
import requests                 # Библиотека работы с HTTP-запросами по API
import json                     # Для обработки полученных результатов запросов
import time                     # Для задержки между запросами
import os                       # Для работы с файлами
import pandas as pd             # Для формирования датафрейма с результатами
import re                       # Для работы с регулярными выражениями
import numpy as np              # Библиотека работы с массивами
import seaborn as sns           # Статистическая визуализация данных
import matplotlib.pyplot as plt # Визуализация данных
import plotly.graph_objs as go
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavyatkina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
## ДАВАЙТЕ БЕЗ ДАВАЙТЕ ###

In [5]:
df = pd.read_csv('data/df.csv')

In [6]:
df

,id,premium,name,has_test,response_letter_required,address,response_url,sort_point_distance,published_at,created_at,...,address.metro_stations,address.id,salary,address.metro.station_name,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,description
0,79110745,False,Data Engineer,False,False,NaN,NaN,NaN,2023-04-10T14:42:13+0300,2023-04-10T14:42:13+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>Ищу <strong>Data Engineer</strong> в SportT...
1,79120615,False,Data engineer,False,True,NaN,NaN,NaN,2023-04-10T17:13:22+0300,2023-04-10T17:13:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<p>В крупную исследовательскую компанию, специ..."
2,78954091,False,Data Engineer / Дата-инженер (Middle),False,False,NaN,NaN,NaN,2023-04-11T15:27:03+0300,2023-04-11T15:27:03+0300,...,"[{'station_name': 'Белорусская', 'line_name': ...",12649470.0,NaN,Белорусская,Кольцевая,5.20,5.0,55.775179,37.582303,"<p><strong><em>Приветствуем тебя, будущий учас..."
3,78934984,False,Data engineer (Стажер),False,False,NaN,NaN,NaN,2023-04-11T11:13:27+0300,2023-04-11T11:13:27+0300,...,"[{'station_name': 'Бауманская', 'line_name': '...",886614.0,NaN,Бауманская,Арбатско-Покровская,3.17,3.0,55.772405,37.679040,<p><strong>НЕ НУЖНО ОТКЛИКАТЬСЯ НА ВАКАНСИЮ! Ч...
4,79084256,False,Data Engineer (lead),False,False,NaN,NaN,NaN,2023-04-13T09:31:35+0300,2023-04-13T09:31:35+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<strong>Что нужно будет делать:</strong> <ul> ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5274,78298565,False,Аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T16:46:50+0300,2023-03-20T16:46:50+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5275,78312389,False,Системный аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5276,78312390,False,Системный аналитик DWH,False,False,NaN,NaN,NaN,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5277,78527633,False,Аналитик систем целевого маркетинга,False,False,NaN,NaN,NaN,2023-03-24T18:40:56+0300,2023-03-24T18:40:56+0300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Проверим, какие столбцы не содержат данные

missing_cols = df.columns[df.isna().all()].tolist()
print(f'Столбцы без данных: {missing_cols}')

Столбцы без данных: []


In [9]:
# Избавимся от них

df = df.drop(missing_cols, axis=1)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5310 entries, 0 to 5309
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  5310 non-null   int64  
 1   id                          5310 non-null   int64  
 2   name                        5310 non-null   object 
 3   snippet.requirement         5281 non-null   object 
 4   snippet.responsibility      5282 non-null   object 
 5   description                 5310 non-null   object 
 6   experience.name             5310 non-null   object 
 7   published_at                5310 non-null   object 
 8   created_at                  5310 non-null   object 
 9   alternate_url               5310 non-null   object 
 10  working_days                5310 non-null   object 
 11  working_time_intervals      5310 non-null   object 
 12  working_time_modes          5310 non-null   object 
 13  accept_temporary            5310 

In [8]:
df.description.isna().sum() 

0

In [9]:
# Подсчитаем количество каждой ваакансии

value_counts = df['name'].value_counts()
result = pd.DataFrame({'name': value_counts.index, 'count':value_counts.values})
result.sort_values(by='count', ascending=False, inplace=True)
print(result)

                                                   name  count
0                                              Аналитик    431
1                                       Бизнес-аналитик    269
2                                    Системный аналитик    188
3                                       Аналитик данных    136
4                                   Маркетолог-аналитик    110
...                                                 ...    ...
1139  Главный финансовый аналитик в дочернюю компани...      1
1138                     Аналитик (складская логистика)      1
1137  Старший Бизнес-Аналитик в Управленческом Конса...      1
1136                             Data Steward (Finance)      1
2675                Аналитик систем целевого маркетинга      1

[2676 rows x 2 columns]


In [16]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from collections import Counter
from bs4 import BeautifulSoup

# Считываем данные из файла и объединяем все значения из столбца 'description'
text = ' '.join([BeautifulSoup(desc, "html.parser").get_text() for desc in df['description'] if isinstance(desc, str)])

# Загрузка стоп-слов
nltk.download('stopwords')
stopwords_ru = set(nltk_stopwords.words("russian"))
stopwords_en = set(nltk_stopwords.words("english"))
stopwords = stopwords_ru | stopwords_en

# Разбиваем текст на слова и удаляем стоп-слова
words = [word.lower() for word in nltk.word_tokenize(text, language='russian') if word.isalpha() and word.lower() not in stopwords]

# Считаем количество уникальных слов
word_count = Counter(words)

# Выводим топ-50 слов
for word, count in word_count.most_common(50):
    print(f"{word}: {count}")



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavyatkina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


работы: 14794
опыт: 12696
данных: 11920
знание: 5755
компании: 5266
анализ: 4447
умение: 3659
работа: 3446
возможность: 3382
требования: 3261
работать: 3030
sql: 3007
условия: 2989
участие: 2809
разработки: 2658
разработка: 2636
понимание: 2481
график: 2458
задачи: 2434
обязанности: 2360
навыки: 2310
рф: 2251
дмс: 2227
подготовка: 2219
анализа: 2184
data: 2173
задач: 2169
excel: 2142
лет: 2135
требований: 2100
аналитика: 2095
образование: 2089
тк: 1967
высшее: 1933
процессов: 1931
предлагаем: 1885
офис: 1865
обучение: 1859
систем: 1829
сотрудников: 1814
python: 1797
отчетности: 1746
оформление: 1743
развития: 1738
команде: 1675
bi: 1646
это: 1583
продаж: 1580
моделей: 1576
решения: 1554


In [17]:
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re

# Загрузка стоп-слов
nltk.download('stopwords')
stopwords_ru = set(stopwords.words("russian"))
stopwords_en = set(stopwords.words("english"))
stopwords = stopwords_ru | stopwords_en

# Инициализация лемматизатора
morph = pymorphy2.MorphAnalyzer()

# Создание пустого списка для хранения навыков и стека
skills = []

# Обработка каждого описания вакансии
for description in df['description']:
    # Преобразование текста в список отдельных слов (токенизация)
    words = [word.lower() for word in re.findall(r'\b\w+\b', description) if len(word) > 1]
    # Удаление стоп-слов
    words = [word for word in words if word not in stopwords]
    # Приведение слов к нормальной форме (лемматизация)
    words = [morph.parse(word)[0].normal_form for word in words]
    # Идентификация навыков и стека
    for word in words:
        if word in ['python', 'javascript', 'react', 'django', 'aws', 'sql', 'html', 'css']:
            skills.append(word)

# Создание словаря с количеством упоминаний каждого навыка/термина
skills_counts = {skill: skills.count(skill) for skill in set(skills)}

# Вывод результатов
for skill, count in sorted(skills_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{skill}: {count}")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavyatkina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


sql: 3476
python: 1968
javascript: 55
html: 51
aws: 45
css: 32
react: 26
django: 19


In [18]:
skills = {
    'Аналитики данных': ['Анализ данных', 'Статистика', 'Машинное обучение', 'SQL', 'Python/R', 'Excel',
                         'BI-системы', 'Визуализация данных', 'Работа с большими объемами данных'],
    'Инженеры данных': ['Базы данных', 'SQL/NoSQL', 'ETL-процессы', 'Python/Scala/Java', 'Hadoop/Spark',
                        'AWS/Azure/GCP', 'Airflow/Luigi', 'Архитектура данных'],
    'Data-сайентисты': ['Машинное обучение', 'Статистика', 'Анализ данных', 'Python/R', 'SQL/NoSQL',
                        'Библиотеки машинного обучения (Scikit-learn, TensorFlow, PyTorch и др.)',
                        'Визуализация данных', 'Работа с большими объемами данных'],
    'Специалисты по визуализации данных': ['Визуализация данных', 'Tableau/Power BI', 'Python/R', 'SQL', 'Инфографика',
                                           'Работа с большими объемами данных', 'UI/UX-дизайн'],
    'Бизнес-аналитики': ['Бизнес-анализ', 'Excel', 'SQL', 'Python/R', 'BI-системы', 'Визуализация данных',
                         'Статистика', 'Работа с большими объемами данных'],
    'Big Data-специалисты': ['Hadoop/Spark', 'SQL/NoSQL', 'Python/Scala/Java', 'AWS/Azure/GCP', 'ETL-процессы',
                             'Архитектура данных', 'Работа с большими объемами данных']
}


In [19]:
import re

def classify_vacancy(description):
    specialization = []
    for key in skills.keys():
        for skill in skills[key]:
            if re.search(skill, description, re.IGNORECASE):
                specialization.append(key)
                break
    return specialization


In [20]:
# Создаем списки с ключевыми словами для каждой специализации
data_analyst_skills = ['Анализ данных', 'Статистика', 'Машинное обучение', 'SQL', 'Python', 'R', 'Excel', 'BI-системы', 'Визуализация данных', 'большие данные']
data_engineer_skills = ['Базы данных', 'SQL', 'NoSQL', 'ETL', 'Python', 'Scala', 'Java', 'Hadoop', 'Spark', 'AWS', 'Azure', 'GCP', 'Airflow', 'Luigi', 'Архитектура данных', 'большие данные']
data_scientist_skills = ['Машинное обучение', 'Статистика', 'Анализ данных', 'Python', 'R', 'SQL', 'NoSQL', 'Scikit-learn', 'TensorFlow', 'PyTorch', 'Визуализация данных', 'большие данные']
data_viz_skills = ['Визуализация данных', 'Tableau', 'Power BI', 'Python', 'R', 'SQL', 'Инфографика', 'большие данные', 'UI', 'UX']
business_analyst_skills = ['Бизнес-анализ', 'Excel', 'SQL', 'Python', 'R', 'BI-системы', 'Визуализация данных', 'Статистика', 'большие данные']
big_data_skills = ['Hadoop', 'Spark', 'SQL', 'NoSQL', 'Python', 'Scala', 'Java', 'AWS', 'Azure', 'GCP', 'ETL', 'Архитектура данных', 'большие данные']

# Создаем новый столбец 'skills', в котором будет список навыков, найденных в описании вакансии
df['skills'] = df['description'].apply(lambda x: [skill for skill in data_analyst_skills if skill.lower() in x.lower()] if 'аналитик' in x.lower() or 'аналитика' in x.lower() else
                                             [skill for skill in data_engineer_skills if skill.lower() in x.lower()] if 'инженер' in x.lower() else
                                             [skill for skill in data_scientist_skills if skill.lower() in x.lower()] if 'сайентист' in x.lower() or 'ученый' in x.lower() else
                                             [skill for skill in data_viz_skills if skill.lower() in x.lower()] if 'визуализатор' in x.lower() or 'дизайнер' in x.lower() else
                                             [skill for skill in business_analyst_skills if skill.lower() in x.lower()] if 'бизнес-аналитик' in x.lower() or 'аналитик бизнеса' in x.lower() else
                                             [skill for skill in big_data_skills if skill.lower() in x.lower()] if 'big data' in x.lower() else [])


In [21]:
df

,Unnamed: 0,id,name,snippet.requirement,snippet.responsibility,description,experience.name,published_at,created_at,alternate_url,...,address.lat,address.lng,address.raw,address.metro.station_name,address.metro.line_name,address.metro.station_id,address.metro.line_id,address.metro.lat,address.metro.lng,skills
0,0,79110745,Data Engineer,Разговорный английский. Опыт работы <highlight...,Взаимодействие с <highlighttext>data</highligh...,<p>Ищу <strong>Data Engineer</strong> в SportT...,От 3 до 6 лет,2023-04-16T14:42:13+0300,2023-04-16T14:42:13+0300,https://hh.ru/vacancy/79110745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SQL, Python, R]"
1,1,79120615,Data engineer,"Высшее техническое образование, предпочтительн...","Системный анализ, инженерная аналитика. Разраб...","<p>В крупную исследовательскую компанию, специ...",От 1 года до 3 лет,2023-04-16T17:13:22+0300,2023-04-16T17:13:22+0300,https://hh.ru/vacancy/79120615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SQL, R]"
2,2,78934984,Data engineer (Стажер),Имеете высшее образование или заканчиваете учи...,По результатам обучения – обязательная защита ...,<p><strong>НЕ НУЖНО ОТКЛИКАТЬСЯ НА ВАКАНСИЮ! Ч...,Нет опыта,2023-04-14T11:13:27+0300,2023-04-14T11:13:27+0300,https://hh.ru/vacancy/78934984,...,55.774284,37.669145,"Москва, Нижняя Красносельская улица, 40/12к2",Бауманская,Арбатско-Покровская,3.17,3.0,55.772405,37.679040,"[SQL, R]"
3,3,78954091,Data Engineer / Дата-инженер (Middle),Понимание и интерес к области больших данных. ...,Мероприятия для поддержания хорошего настроени...,"<p><strong><em>Приветствуем тебя, будущий учас...",От 1 года до 3 лет,2023-04-14T15:27:03+0300,2023-04-14T15:27:03+0300,https://hh.ru/vacancy/78954091,...,55.773393,37.595995,"Москва, 4-я Тверская-Ямская улица, 14с1",Белорусская,Кольцевая,5.20,5.0,55.775179,37.582303,"[SQL, ETL, Python, Hadoop, Spark, Airflow]"
4,4,79084256,Data Engineer (lead),"Хорошее знание технологий из стека: Python, SQ...",Готовить данные для моделей машинного обучения...,<strong>Что нужно будет делать:</strong> <ul> ...,От 3 до 6 лет,2023-04-16T09:31:35+0300,2023-04-16T09:31:35+0300,https://hh.ru/vacancy/79084256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[SQL, Python, R]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5305,5305,78300415,Системный аналитик DWH,"Опыт в выявлении, сборе и формализации требова...",Анализ алгоритмов реализации PnL отчетности в ...,<p><strong>IBS строит долгосрочные партнерства...,От 3 до 6 лет,2023-03-20T17:11:03+0300,2023-03-20T17:11:03+0300,https://hh.ru/vacancy/78300415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5306,5306,78298565,Аналитик DWH,"Опыт в выявлении, сборе и формализации требова...",Анализ алгоритмов реализации PnL отчетности в ...,<p><strong>IBS строит долгосрочные партнерства...,От 3 до 6 лет,2023-03-20T16:46:50+0300,2023-03-20T16:46:50+0300,https://hh.ru/vacancy/78298565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5307,5307,78312389,Системный аналитик DWH,"Опыт в выявлении, сборе и формализации требова...",Анализ алгоритмов реализации PnL отчетности в ...,<p><strong>IBS строит долгосрочные партнерства...,От 3 до 6 лет,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,https://hh.ru/vacancy/78312389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5308,5308,78312390,Системный аналитик DWH,"Опыт в выявлении, сборе и формализации требова...",Анализ алгоритмов реализации PnL отчетности в ...,<p><strong>IBS строит долгосрочные партнерства...,От 3 до 6 лет,2023-03-20T21:23:22+0300,2023-03-20T21:23:22+0300,https://hh.ru/vacancy/78312390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [30]:
import http.client


In [42]:
def process_vacancies(vacancy_ids):
    headers = {"User-Agent": "hh-recommender"}
    for vac_id in vacancy_ids:
        conn = http.client.HTTPSConnection("api.hh.ru")
        conn.request("GET", "/vacancies/{}".format(vac_id), headers=headers)
        r1 = conn.getresponse()
        data1 = r1.read()
        vacancy = json.loads(data1)
        
        # title
        p_title = ''
        title = re.sub('[^a-zа-я]+', ' ', vacancy['name'].lower(), re.UNICODE)
        words = re.split(r'\s{1,}', title.strip())
        for title_word in words:
            title_word = stemmer.stem(title_word.strip())
            if len(title_word.strip()) > 1:
                p_title = p_title + " " + title_word

        # description
        p_doc = ''
        doc = re.sub('<[^<]+?>', ' ', vacancy['description'], re.UNICODE)
        words = re.split(r'\s{1,}', doc.strip())
        for word in words:
            word = stemmer.stem(word.strip())
            if len(word.strip()) > 1:
                p_doc = p_doc + " " + word

        print('Vacancy id:', vac_id)
        print('Title:', p_title)
        print('Description:', p_doc)
        print('--------')



        #title
        p_title = ''
        title = re.sub('[^a-zа-я]+', ' ', vacancy['name'].lower(), re.UNICODE)
        words = re.split(r'\s{1,}', title.strip())
        for title_word in words:
            title_word = stemmer.stem(title_word)
            if len(title_word.strip()) > 1:
                p_title = p_title + " " + title_word.strip()

        #keyskills
        p_skills = ''
        vac_skills = vacancy['key_skills']
        for skill in vac_skills:
            words = re.split(r'\s{1,}', skill['name'].lower().strip())
            for word in words:
                word = stemmer.stem(word)
                if len(word.strip()) > 1:
                    p_skills = p_skills + " " + word.strip()
                    
        #salary
        salary = None
        if vacancy['salary'] != None:
            if vacancy['salary']['from'] == None and vacancy['salary']['to'] != None:
                salary = vacancy['salary']['to']/currency_rates[vacancy['salary']['currency']]
            elif vacancy['salary']['to'] == None and vacancy['salary']['from'] != None:
                salary = vacancy['salary']['from']/currency_rates[vacancy['salary']['currency']]
            elif vacancy['salary']['to'] != None and vacancy['salary']['from'] != None:
                salary = ((vacancy['salary']['from'] + vacancy['salary']['to'])/2)/currency_rates[vacancy['salary']['currency']]
        max_salary = 500000.0
        if salary >= max_salary:
            salary = max_salary

        try:
            area_id = areas_map[vacancy['area']['id']]
        except KeyError:
            print('missed area id {} for vacancy {}'.format(vacancy['area']['id'], vacancy['id']))
            
            conn = http.client.HTTPSConnection("api.hh.ru")
            conn.request("GET", "https://api.hh.ru/vacancies/{}".format(vacancy['id']), headers=headers)
            r1 = conn.getresponse()
            if r1 != 200:
                conn.close()
                conn = http.client.HTTPSConnection("api.hh.ru")
                conn.request("GET", "https://api.hh.ru/vacancies/{}".format(vacancy['id']), headers=headers)
                r1 = conn.getresponse()
            missed_area_vacancy = r1.read()
            conn.close()
            missed_area_vacancy_json = json.loads(missed_area_vacancy)
            
            conn = http.client.HTTPSConnection("api.hh.ru")
            conn.request("GET", "https://api.hh.ru/areas/{}"
                         .format(missed_area_vacancy_json['area']['id']), headers=headers)
            r1 = conn.getresponse()
            missed_area = r1.read()
            conn.close()
            missed_area_json = json.loads(missed_area)
            areas_map[vacancy['area']['id']] = missed_area_json['parent_id']
            area_id = missed_area_json['parent_id']

        p_doc = p_doc + " " + p_title + " " + p_skills
        
        #specializations
        specializations = set()
        try:
            if vacancy['specializations'] != None:
                for spec in vacancy['specializations']:
                    specializations.add(spec['profarea_id'])
        except KeyError:
            print('cant find specialization')

        feature_p_doc = count_vectorizer.transform([p_doc])
        tfidf_feature_p_doc = tfidf_transformer.transform(feature_p_doc)
            
        data = {}
        data['features'] = json.dumps(tfidf_feature_p_doc.toarray()[0].tolist()).encode("zlib")
        data['salary'] = salary
        data['area'] = area_id
        data['specializations'] = specializations
        r.hmset(vacancy['id'], data)
        r.expire(vacancy['id'], timeout)

In [50]:
def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""

    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                description = data['description']
                time.sleep(0.5)
                break
            elif response.status_code == 403:
                raise ValueError("Требуется решить капчу. Прекращение выполнения функции.")
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 0.2 секунды.")
            time.sleep(0.2)
            continue
    
    return description

In [51]:
get_description(78954091)




ValueError: Требуется решить капчу. Прекращение выполнения функции.

In [57]:
import requests
import webbrowser

def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}?host=hh.ru'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['description']
    else:
        webbrowser.open(
'https://hh.ru/account/captcha?state=123abc&backurl=https://hh.ru')
        response.raise_for_status()

vacancy_id = '78954091' # заменить на нужный номер вакансии

get_description(vacancy_id)

HTTPError: 403 Client Error: Forbidden for url: https://api.hh.ru/vacancies/78954091?host=hh.ru

In [49]:
import requests
import pandas as pd
import numpy as np


res = []
for indx, skill in enumerate(skills):
    print(f'\ncollecting <{skill}> ({indx+1} of {len(skills)})')
    for page in range(pages):
        params = {
            'text': f'{skill}',
            'page': page,
            'per_page': 100,
        }
        req = requests.get('https://api.hh.ru/vacancies', params).json()
        if 'items' in req.keys():
            res.extend(req['items'])
        print('|', end='')

df = pd.DataFrame(res)

df = df[['id', 'name', 'salary', 'schedule', 'accept_temporary', 'area', 'published_at']]


df["city"] = (df["area"]
                    .apply(lambda x: x.get("name")
                                    if isinstance(x, dict)
                                    else np.nan))
df["schedule"] = (df["schedule"]
                    .apply(lambda x: x.get("id")
                                     if isinstance(x, dict)
                                    else np.nan))
df["salary_from"] = (df["salary"]
                      .apply(lambda x: x.get("from")
                                    if isinstance(x, dict)
                                    else np.nan))
df["salary_to"] = (df["salary"]
                      .apply(lambda x: x.get("to")
                                    if isinstance(x, dict)
                                    else np.nan))
df["salary_currency"] = (df["salary"]
                    .apply(lambda x: x.get("currency")
                                    if isinstance(x, dict)
                                    else np.nan))

df = (df[df['salary_currency'] == 'RUR']
        .drop(['salary', 'area'], axis=1)
        .drop_duplicates()
        )
df['salary_from'] = df['salary_from'].fillna(df['salary_to'])
df['salary_to'] = df['salary_to'].fillna(df['salary_from'])
df = df.dropna(subset=['salary_from', 'salary_to'])
df['salary_mean'] = round((df['salary_from'] + df['salary_to']) / 2)
df = df.drop(['salary_currency', 'salary_from', 'salary_to'], axis=1)

features = {
    'skills': [],
    'experience': [],
    'professional roles': [],
    'employer': [],
    'employer trusted': [],
    'url': [],
}
print('\nskill parcing...')

all_skills = []

for id in df['id']:
    req = requests.get(f'https://api.hh.ru/vacancies/{id}').json()
    skills = [skill['name'] for skill in req['key_skills']]
    features['skills'].append(skills)
    all_skills.extend(skills)
    features['experience'].append(req['experience']['name'])
    features['professional roles'].append(req['professional_roles'][0]['name'])
    features['employer'].append(req['employer']['name'])
    features['employer trusted'].append(req['employer']['trusted'])
    features['url'].append(req['alternate_url'])
    print('|', end='')
print()

df['skills'] = features['skills']
df['experience'] = features['experience']
df['professional roles'] = features['professional roles']
df['employer'] = features['employer']
df['employer trusted'] = features['employer trusted']
df['url'] = features['url']

print(df.info())

df.to_csv(filename, index=False)
print(f'data stored in file {filename}')


collecting <Аналитики данных> (1 of 6)


NameError: name 'pages' is not defined

In [46]:
skill_list = ['machine AND learning', 'data AND science', 'NLP',
              'spark', 'hadoop', 'pandas', 'dask', 'deep AND learning', 'pytorch',
              'tensorflow', 'keras', 'ai AND developer', 'computer AND vision',
              'нейронные AND сети', 'big AND data']

get_vacancies(skill_list, 'data.csv')


collecting <machine AND learning> (1 of 15)
||||||||||
collecting <data AND science> (2 of 15)
||||||||||
collecting <NLP> (3 of 15)
||||||||||
collecting <spark> (4 of 15)
||||||||||
collecting <hadoop> (5 of 15)
||||||||||
collecting <pandas> (6 of 15)
||||||||||
collecting <dask> (7 of 15)
||||||||||
collecting <deep AND learning> (8 of 15)
||||||||||
collecting <pytorch> (9 of 15)
||||||||||
collecting <tensorflow> (10 of 15)
||||||||||
collecting <keras> (11 of 15)
||||||||||
collecting <ai AND developer> (12 of 15)
||||||||||
collecting <computer AND vision> (13 of 15)
||||||||||
collecting <нейронные AND сети> (14 of 15)
||||||||||
collecting <big AND data> (15 of 15)
||||||||||
skill parcing...
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

KeyError: 'key_skills'

In [47]:
req

NameError: name 'req' is not defined